In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load data
df = pd.read_csv("data/train100k.csv")

In [3]:
# convert to datetime format
df.date = pd.to_datetime(df.date)
# parse the strings to itemids
df.itemids = df.itemids.apply(lambda row: [int(i) for i in row.split(" ")])

In [4]:
# explore data
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711877 entries, 0 to 1711876
Data columns (total 3 columns):
 #   Column   Dtype         
---  ------   -----         
 0   userid   int64         
 1   date     datetime64[ns]
 2   itemids  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 39.2+ MB


,userid,date,itemids
0,7226385,2019-01-22,"[42203, 41183, 15823, 39620]"
1,7226385,2019-02-12,"[54231, 14939, 39462]"
2,7226385,2019-03-11,"[15823, 21028, 39620, 52846]"
3,7226385,2019-04-03,"[14939, 39620, 27542, 21028, 19353]"
4,7226385,2019-05-23,"[21028, 21028, 14939, 15823]"


In [5]:
# feather format preserves dtypes of columns
# df.to_feather("data/train100k.feather")

In [6]:
# df = pd.read_feather("data/train100k.feather")

In [7]:
# load test DF + convert col to datetime format
testDf = pd.read_csv("data/test24k.csv")
testDf.date = pd.to_datetime(testDf.date)
testUserIds = set(testDf.userid.unique())
testDf.info()
testDf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   userid   24000 non-null  int64         
 1   date     24000 non-null  datetime64[ns]
 2   itemids  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 562.6 KB


,userid,date,itemids
0,6568788,2020-01-15,NaN
1,2771293,2020-01-30,NaN
2,6761196,2020-01-29,NaN
3,3711380,2020-01-23,NaN
4,9576135,2020-01-16,NaN


In [8]:
len(testUserIds)

24000

### Baseline - Previous Purchase

In [9]:
# example
exampleUserId = testDf.sample().userid.iloc[0]
display(df[df.userid == exampleUserId].sort_values("date").tail(6))

,userid,date,itemids
1520731,8376293,2019-09-03,"[59811, 12996, 32332, 21098, 15633, 24518, 46056]"
1520732,8376293,2019-09-24,"[38999, 53952, 44195, 58234, 23433, 23433, 157..."
1520733,8376293,2019-10-18,"[21098, 38786, 58234, 30723, 34720]"
1520734,8376293,2019-11-29,"[25980, 12996, 56015, 21098, 26814]"
1520735,8376293,2019-12-17,"[43437, 41755, 44453, 25849, 58234, 34381]"
1520736,8376293,2020-01-21,"[21098, 34720, 58234, 53952]"


Now let's get the last purchase for all of the test users

In [10]:
# obtain data of testing users
test = df[df.userid.isin(testUserIds)].reset_index(drop=True)
# train = df[~df.userid.isin(testUserIds)].reset_index(drop=True)

In [11]:
# get last purchase for each userid
lastPurchaseDf = test.iloc[test.groupby("userid").date.idxmax()]
lastPurchaseDf.head(3)

,userid,date,itemids
186844,1000167,2020-01-15,"[56638, 14028, 21645, 32306, 25940, 25940, 58951]"
63867,1000630,2020-08-18,[45033]
243677,1000648,2020-02-18,"[16059, 31856, 56015, 47049, 52206, 55503]"


In [12]:
testDf[['userid', 'date']].head(2)

,userid,date
0,6568788,2020-01-15
1,2771293,2020-01-30


In [13]:
# transform data to submission format and save
submission = testDf[['userid', 'date']].merge(lastPurchaseDf[['userid', 'itemids']], left_on='userid', right_on='userid').copy()
display(submission.head(2))

submission.itemids = submission.itemids.apply(lambda row: " ".join(str(itemid) for itemid in row))
display(submission.head(2))

submission.to_csv("submission-last-purchase.csv", index=False)

,userid,date,itemids
0,6568788,2020-01-15,"[19240, 32367, 31315]"
1,2771293,2020-01-30,"[52979, 28998]"


,userid,date,itemids
0,6568788,2020-01-15,19240 32367 31315
1,2771293,2020-01-30,52979 28998


Now we could simply submit the last purchase. But there is a catch. Ideally we would like to know how well the model performs. To do that, we need to create our own evaluation (also called validation) dataset. 

# Measuring our algorithm - Jaccard similarity

In the first round, we are using a score that ignores the counts of the item in the target basket.

In [ ]:
def jaccard_simple(list1, list2) -> float:
    """First round score that ignores the counts of the items."""
    set1 = set(list1)
    set2 = set(list2)
    
    # Also case of empty == empty
    if set1 == set2:
        return 1

    intersec = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersec / union if union > 0 else 0

```python
>>> jaccard_simple([1, 2, 3], [1]) 
0.3333333333333333
>>> jaccard_simple([1], [1, 2, 3]) 
0.3333333333333333
>>> jaccard_simple([1, 2, 3], [1, 1, 1])
0.3333333333333333
>>> jaccard_simple([1, 2, 3], [4])
0.0
```

# Validation dataset
The goal of machine learning is, generally speaking, to create a model tha generalizes well across different (and most importantly unseen) data. To obtain an unbiased estimate of a model's performance, we traditionally use a separate dataset to train a model (`train`), and to measure its performance (`validation`). Your client then has a separate `test` dataset which is not shared with you. 

In case of this challenge, we have the list of targets that we want to predict with an accuracy as high as possible (the `test`). But we can submit our efforts only once per hour. This limits us from running many experiments locally, seeing their results and quickly adjusting our algorithms.

To measure this, we will create our own, **validation** dataset. Let's examine our targets now.

In [ ]:
fig, ax = plt.subplots()
targets.groupby(by=[targets.date.dt.year, targets.date.dt.month]).userid.count()\
    .plot(kind="bar", title="Number of target purchases in a given month")
ax.set_xlabel("month")
ax.set_ylabel("number of purchases")
plt.show()

It is worth noting that your validation dataset should be similar to the provided test dataset. What you want is to get as relevant and representative score as possible. We have several options for a validation dataset
- create a completely random sample of the users and select a random purchase from their history
- create a random sample that has similar distribution of target dates as our test targets
- use the last purchase from the given test dataset as our validation (lazy approach)

We will go with the lazy approach here, but it is up to your judgement how to validate your model. A price we will pay for this lazy approach is that we will lower the amount of previous purchases that were guaranteed to be at least 5 to 4. There is no free lunch.

Also, there is a question of whether you should include the validation users in your training dataset or not. That's up to your decision.

In [ ]:
# Get the last purchase (maximum date) for each userid
validation = test.iloc[test.groupby("userid").date.idxmax()].copy()

We can see that by being lazy we get somewhat similar distribution over months. Let's hope it represents the true testing data well enough.

In [ ]:
fig, ax = plt.subplots()
validation.groupby(by=[validation.date.dt.year, validation.date.dt.month]).userid.count()\
    .plot(kind="bar", title="Number of validation purchases in a given month")
ax.set_xlabel("month")
ax.set_ylabel("number of purchases")
plt.show()

In [ ]:
# All the test users data except the validation purchase
# Not that there is also a whole train dataset above
remaining = test.loc[test.index.difference(validation.index)].reset_index(drop=True)

# Measuring the last purchase strategy on the validation dataset

The `remaining` DataFrame now contains only the validation users. So we don't have to subsample it any further.

In [ ]:
pred = remaining.loc[remaining.groupby("userid").date.idxmax()]

Let's study the case of userid `1000648`. 

In [ ]:
val_1000648 = validation[validation.userid == 1000648]
val_1000648

In [ ]:
pred_1000648 = pred[pred.userid == 1000648]
pred_1000648

Let's see how similar the 2 baskets are

In [ ]:
jaccard_simple(pred_1000648.itemids.values[0], val_1000648.itemids.values[0])

Now let's see the results for **all** validation users and average them.

In [ ]:
merged = validation.merge(pred, on="userid", suffixes = ("_val", "_pred"))

In [ ]:
merged["jaccard"] = merged.apply(lambda x: jaccard_simple(x.itemids_val, x.itemids_pred), axis=1)

We get the (macro) averaged jaccard similarity over our validation dataset:

In [ ]:
merged.jaccard.mean()

# Beyond the baseline
The baseline is quite naive. We weren't using most of our data anyway. We could do better! Let's take the full dataset except our validation targets.

In [ ]:
full = pd.concat([train, remaining])

In collaborative filtering, we use the user-item matrix which is a matrix with users in rows and items in columns. If a user had purchased an item, given user-item cell in the matrix is non-zero. 

In [ ]:
unique_items = full.itemids.explode().unique()
unique_users = full.userid.unique()

In [ ]:
# Create mappings between user (and item) id's and matrix indexes
user2index = dict(zip(unique_users, range(len(unique_users))))
index2user = dict(zip(range(len(unique_users)), unique_users))
item2index = dict(zip(unique_items, range(len(unique_items))))
index2item = dict(zip(range(len(unique_items)), unique_items))

We will try a strategy where we will create one matrix for all previous purchases and one matrix for the last purchase. We use the `prev_matrix` for user purchasing similarity and `last_matrix` to get the prediction for the given user.

In [ ]:
# Add a flag whether this purchase was the last one
full["last_purchase"] = full.groupby("userid").date.transform('max') == full.date

In [ ]:
# Build the user-item matrixes
prev_matrix = np.zeros((len(unique_users), len(unique_items)), dtype="int16")
last_matrix = np.zeros((len(unique_users), len(unique_items)), dtype="int16")

# This may take a while and is memory intensive
for row in full.explode(column="itemids").itertuples():
    if row.last_purchase:
        last_matrix[user2index[row.userid], item2index[row.itemids]] += 1
    else:
        prev_matrix[user2index[row.userid], item2index[row.itemids]] += 1

In [ ]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=16, metric="cosine", n_jobs=-1)
neigh.fit(prev_matrix)

In [ ]:
# Map the urse ids into matrix indexes and select the validation users
validation["user_index"] = validation.userid.map(user2index)
validation_users = prev_matrix[validation.user_index.values]

In [ ]:
validation_targets = validation.itemids.to_list()

In [ ]:
# Compute the k nearest neighbours for the validation users
distances, indexes = neigh.kneighbors(validation_users)

In [ ]:
exclude_self = 1

In [ ]:
for i, knn in enumerate([2, 4, 6, 8, 10, 12, 16]):
    # Do not include the closest neighbour (self)
    n_distances = distances[:, exclude_self:knn]
    n_indexes = indexes[:, exclude_self:knn]

    # Invert the distance into weight
    n_distances = 1 - n_distances

    # Normalize the distances by broadcasting
    sums = n_distances.sum(axis=1)
    n_distances = n_distances / sums[:, np.newaxis]

    # Do the prediction
    n_neigh_last = last_matrix[n_indexes]

    # Variable threshold round (you can play with the threshold)
    preds = (n_neigh_last * n_distances[:, :, np.newaxis]).sum(axis=1)
    preds = np.where(preds > 0.5, 1, 0)

    # Get the item ids from the matrix indexes
    nonzero_preds = [np.flatnonzero(row) for row in preds]
    transformed_preds = [[index2item[idx] for idx in row] for row in nonzero_preds]

    score = np.mean([jaccard_simple(pred, target) for pred, target in zip(transformed_preds, validation_targets)])
    print(f"knn: {knn}\t| {score}")